# 1-factor Vasicek Model

This notebook evaluates the 1-factor Vasicek Model as the stochastic process driving the 'short rate'. From this process we can derive an explicit formula for the price of a zero-coupon bond. Moreover, we will use the process to price an interest rate derivative. 

### The Model

Let the short-rate $r_t$ be governed by a process which can be described by the following stochastic differential equation (SDE):

$$ dr_t = \kappa (\mu - r_t)dt + \sigma d\text{W}_t^\mathcal{Q},$$

where $\kappa$ is the a coefficient governing the mean-reversion. The mean or long-term average rate is $\mu$. We denote 'the change' in a variable by using $d$ in front. The process is a Wiener process which means that $\text{W}_t^\mathcal{Q}$ follows a standard normal distribution, under the risk neutral measure  $\mathcal{Q}$. The volatility of the interest rate is characterised by $\sigma$. 

### Bond Pricing 

The value of a zero-coupon bond (ZCB), i.e. a discount bond, with maturity date T (in years) can be expressed at time $t$, under the no-arbitrage assumption as 

$$P(t, T) = A(t, T) e^{-B(t, T) r_t}, $$

which is exponential affine in the interest rate. Or, the log of the bond price is linearly related to the interest rate. 

Here, we have $A(t,T)$ and $B(t, T)$ which are defined as follows

$$ B(t, T) = \frac{1 - e^{\kappa (T - t)}}{\kappa} $$

$$ A(t, T) = \exp \left\{ \left( \mu - \frac{\sigma^2}{2\kappa^2} \right) [B(t,T) - (T-t)] - \frac{\sigma^2}{4\kappa} B^2(t,T) \right\} $$

Note: the long-term variance of the process is $\frac{\sigma^2}{2\kappa^2}$ which is explicitly in the formula for $B(t,T)$.

In [39]:
# libraries
import numpy as np

kappa = 0.8
mu = 0.05
r_0 = mu
sigma = 0.02
lambda_ = 0
T = 3
t = 0


In [40]:
# get the bond price 

def getATSMPrice(r_t, t, T, kappa, mu, sigma):
    B = ( 1 - np.exp( kappa * (T - t) ) ) / kappa
    A = np.exp( ( mu - sigma**2 / (2*kappa**2) ) * ( B - (T-t) ) - sigma**2 / (4*kappa) * B**2 )
    P = A * np.exp( - B*r_t )
    return P



In [41]:
print(getATSMPrice(mu, t, T, kappa, mu, sigma))

0.8480895186006085


### Bond price through Riemann sum approximation

Still, consider $P(t,T)$ the price of ZCB. Under the risk neutral measure it holds that 

$$ P(t,T) = \mathbb{E} \left[ e^{-\int_t^T r_u du} * 1 | \mathcal{F}_t \right], $$

where 1 denotes the payoff at time $T$, and $\mathcal{F}_t$ is the information set up to time $t$. 

We can numerically integrate to get an expression for $\int_t^T r_u du$, by approximating this expression using a Riemann sum as

$$ \int_t^T r_u du \approx \sum_{i=0}^{n-1} r_{t+\frac{i(T-t)}{n}} * \frac{T-t}{n}, $$

where $(T-t)$ is artificially split up in $n$ 'units'. To match the Euler grid, we must have $n \leq \frac{T-t}{\Delta}$. 

Note that we have an expression for the short-rate in terms of its SDE. Luckily, we have a closed-form solution:

$$ r_{t+\Delta} = e^{-\kappa \Delta} r_t + (1 - e^{-\kappa \Delta}) \mu + z_{t+\Delta}, $$

with $z_{t+\Delta} \sim N(0, \sigma^2 (1 - e^{-2\kappa \Delta}) / (2\kappa))$.


In [ ]:
# get simulated bond price (using Euler discretisation and Riemann sums)

kappa = 0.8
T = 1
mu = 0.05
r_0 = mu
sigma = 0.02

delta = 1/10
nSimulations = 10

def getSimulatedBondPrice(kappa, mu, sigma, r_0, nSimulations, delta, T):

    nSteps = int(T / delta) # number of steps on grid
    varSR = sigma**2 * (1 - np.exp( -2 * kappa * delta )) / (2*kappa) # short-rate variance

    ratePaths = np.zeros([nSimulations, nSteps])
    payoff = np.zeros(nSimulations)

    for i in range(nSimulations):

        # initiate path from r_0
        ratePaths[i, 0] = r_0 

        for t in range(1, nSteps):
            # generate disturbance
            z_td = np.random.normal(0, np.sqrt(varSR))

            # closed-form solution for r_{t+d}
            ratePaths[i, t] = np.exp( - kappa * delta ) * ratePaths[i, (t-1)] + (1 - np.exp( - kappa * delta )) * mu + z_td

        payoff[i] = np.exp( - np.sum(ratePaths[i, ]) * delta )

    price = np.mean(payoff)

    return price, ratePaths


### Euler Discretisation

Alternatively, we can use an Euler discretisation, which gives 

$$ r_{t+\Delta} = r_t + \kappa (\mu - r_t)\Delta +\sigma z_{t+\Delta} , $$

with $z_{t+\Delta} = W^{\mathcal{Q}}_{t+\Delta} - W^{\mathcal{Q}}_{t}  \sim N(0, \Delta)$. The Euler Discretisation is general, and applies also to, e.g. CIR.

If we also numerically integrate the short-rate but using above formula, we get:

In [44]:
# get simulated bond price (using Euler discretisation and Riemann sums)

kappa = 0.8
T = 1
mu = 0.05
r_0 = mu
sigma = 0.02

delta = 1/10
nSimulations = 10

def getSimulatedBondPrice2(kappa, mu, sigma, r_0, nSimulations, delta, T):

    nSteps = int(T / delta) # number of steps on grid
    varSR = delta # short-rate variance

    ratePaths = np.zeros([nSimulations, nSteps])
    payoff = np.zeros(nSimulations)

    for i in range(nSimulations):

        # initiate path from r_0
        ratePaths[i, 0] = r_0 

        for t in range(1, nSteps):
            # generate disturbance
            z_td = np.random.normal(0, np.sqrt(varSR))

            # closed-form solution for r_{t+d}
            r_t + kappa * ( mu - r_t ) * delta + sigma * z_td
            ratePaths[i, t] = ratePaths[i, (t-1)] + kappa * ( mu - ratePaths[i, (t-1)] ) * delta + sigma * z_td

        payoff[i] = np.exp( - np.sum(ratePaths[i, ]) * delta )

    price = np.mean(payoff)

    return price, ratePaths


In [ ]:
# try out a few different step sizes and number of simulations
delta_grid = np.array([1/10, 1/100, 1/1000])
nSimulations_grid = np.array([10, 100, 1000])
bondPrices = np.zeros([3, 3])
bondPrices2 = np.zeros([3, 3])

for i, delta in enumerate(delta_grid):
    for j, nSimulations in enumerate(nSimulations_grid):
        bondPrices[i,j], ratePaths = getSimulatedBondPrice(kappa, mu, sigma, r_0, nSimulations, delta, T)
        bondPrices2[i,j], ratePaths2 = getSimulatedBondPrice2(kappa, mu, sigma, r_0, nSimulations, delta, T)

print("Using first representation")
print(bondPrices)
print("Using second representation")
print(bondPrices2)

[[0.95331999 0.95215592 0.95100943]
 [0.94684859 0.95202873 0.95116562]
 [0.9484813  0.9507232  0.95114055]]
[[0.94959619 0.95159286 0.95134529]
 [0.94899195 0.9496393  0.95156069]
 [0.95270683 0.94944263 0.9512564 ]]


### Short-rate paths visualised

Let's graph the short-rate paths first using a rather large stepsize, and second a smaller step size. As we saw in the previous table, we tend to converge to a more 'accurate' price given our assumptions, as we increase the number of steps (decrease the step size) and as we increase the number of simulations. Note, however, that we are converging towards our prior set of beliefs about the underlying process, and not necessarily towards the true process 😎.

In [38]:
import plotly.graph_objects as go
import pandas as pd
delta = 1/10
nSimulations = 15 
pricesChart, ratePathsChart = getSimulatedBondPrice(kappa, mu, sigma, r_0, nSimulations, delta, T)
ratePathsChart = ratePathsChart.T

ratePathsChart = pd.DataFrame(ratePathsChart)
fig = go.Figure()

for col in ratePathsChart.columns:
    fig.add_trace(go.Scatter(
        x=ratePathsChart.index*delta,
        y=ratePathsChart[col],
        mode='lines',
        name=str(col),
        line=dict(width=1)
    ))
            
fig.update_layout(
    title=f'Simulated short-rate paths', 
    xaxis_title='Time', 
    yaxis_title='Rate',
    showlegend=False,
    height=600,
    width=1000
)

fig.show()

In [37]:
delta = 1/1000
nSimulations = 15 
pricesChart, ratePathsChart = getSimulatedBondPrice(kappa, mu, sigma, r_0, nSimulations, delta, T)
ratePathsChart = ratePathsChart.T

ratePathsChart = pd.DataFrame(ratePathsChart)
fig = go.Figure()

for col in ratePathsChart.columns:
    fig.add_trace(go.Scatter(
        x=ratePathsChart.index*delta,
        y=ratePathsChart[col],
        mode='lines',
        name=str(col),
        line=dict(width=1)
    ))
            
fig.update_layout(
    title=f'Simulated short-rate paths', 
    xaxis_title='Time', 
    yaxis_title='Rate',
    showlegend=False,
    height=600,
    width=1000
)

fig.show()

### Pricing a bond option

Now that we have our model for the short-rate, we can use this model to price a European call option (i.e. can only be exercised at expiration) on a bond. In essence, we can just copy what we did before with simulating short-rate paths and their corresponding bond prices. Subsequently, we evaluate payoff of the call, which pays 0 if bond price below strike at expiration, and pays the difference between bond price and strike price at expiration if bond price is greater. 

Rigorously, let $T_e$ be the date of expiration of the call. Let $T_m$ be the maturity date of the bond. The strike is denoted as $K$. For the call we are interested in the following payoff:

$$ \left[ P_{T_e}(T_m - T_e) - K \right]^{+},$$

and since we are at time $t$ we'll have to evaluate this payoff only using information up to time $t$. Consequently, our European call price is defined as

$$ p_t^{EUCall} = \mathbb{E}^{\mathcal{Q}} \left[ e^{-\int_t^{T_e} r_u du} \left[ P_{T_e}(T_m - T_e) - K \right]^{+} | \mathcal{F}_t \right], $$

where $\mathcal{F}_t$ is the filtration/information set up to time $t$. 

In [51]:
# Get simulated bond option price
kappa = 0.8
mu = 0.05
r_0 = mu
sigma = 0.02

T_e = 0.5
T_m = 1
K = 0.92

delta = 1/10
nSimulations = 10

# def getSimulatedBondPrice(kappa, mu, sigma, r_0, nSimulations, delta, T):

nSteps = int(T_e / delta) # number of steps on grid until expiry
varSR = sigma**2 * (1 - np.exp( -2 * kappa * delta )) / (2*kappa) # short-rate variance

ratePathsBeforeExpiry = np.zeros([nSimulations, nSteps])
bondPrice = np.zeros(nSimulations)
payoff = np.zeros(nSimulations)

for i in range(nSimulations):

    # initiate path from r_0
    ratePaths[i, 0] = r_0 

    for t in range(1, nSteps):
        # generate disturbance
        z_td = np.random.normal(0, np.sqrt(varSR))

        # closed-form solution for r_{t+d}
        ratePathsBeforeExpiry[i, t] = np.exp( - kappa * delta ) * ratePaths[i, (t-1)] + (1 - np.exp( - kappa * delta )) * mu + z_td

    rate_T_e = ratePathsBeforeExpiry[i, -1]

    bondPrice[i], ratePathsAfterExpiry = getSimulatedBondPrice(kappa, mu, sigma, rate_T_e, nSimulations, delta, (T_m - T_e)) 

    # first part of payoff is discounting from T_e to t, second part is the expected value of option at T_e
    payoff[i] = np.exp( - np.sum(ratePathsBeforeExpiry[i, ]) * delta ) * np.max([ ( bondPrice[i] - K ), 0])


callPrice = np.mean(payoff) 



print(callPrice)
    # return price, ratePaths

0.05254303171609055
